**Data preparation**

In [1]:
import warnings
warnings.simplefilter(action='ignore')

import os
import arcpy
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [2]:
wks= r'Q:\dss_workarea\mlabiadh\workspace\20250505_t4w_hctf_updates'
gdb = r'Q:\projects\Mflnro\Together_for_Wildlife_Funding_Dashboard\outputs\final_data.gdb'

In [3]:
gdf_pnts = gpd.read_file(
    gdb,
    layer='T4W_Goal_Action_points_with_Attributes',
    driver='OpenFileGDB',
)

df_pnts= gdf_pnts.drop(columns=['geometry'])

In [8]:
df =  pd.read_excel(
    os.path.join(wks, 'May20th2025_T4W_Goal_Actions_HCTF_edited.xlsx'),
)
df= df.drop(columns=['Comments'])

In [9]:
# This checks if the Project_ID already starts with a year pattern (4 digits followed by a dash)
condition = ~df['Project_ID'].str.match(r'^\d{4}-')

# Apply the concatenation only to records where the Project_ID doesn't already have year info
df.loc[condition, 'Project_ID'] = df.loc[condition, 'Year'].astype(str) + '-' + df.loc[condition, 'Project_ID'].astype(str)


df['Action'] = df['Action'].replace(
    'Action 9 - On-the-ground',
    'Action 09 - On-the-ground Action'
)

In [10]:
mask = (
    df_pnts['Project_ID'].str.lower().str.contains('hctf', na=False)
    | df_pnts['Project_Type'].eq('HCTF')
)

df_pnts_hctf = df_pnts[mask]

In [14]:
#missing projects
missing_ids = df_pnts_hctf.loc[~df_pnts_hctf['Project_ID'].isin(df['Project_ID']), 'Project_ID'].dropna().unique()
list(missing_ids)

['2024-T4W-HCTF39', '2024-T4W-HCTF38']

In [15]:
geometry = [Point(xy) for xy in zip(df['Longitude'], df['Latitude'])]

gdf = (
    gpd.GeoDataFrame(
        df,
        geometry=geometry,
        crs="EPSG:4326"
    )
    .to_crs("EPSG:3005")
)

**Update the Points featureclass**

In [16]:
#fc = r'Q:\dss_workarea\mlabiadh\workspace\20250505_t4w_hctf_updates\test_data.gdb\t4w_points_data_test'
fc = os.path.join(gdb,'T4W_Goal_Action_points_with_Attributes')

key='Project_ID'

# index GeoDataFrame by key for fast lookups
gdf_indexed = gdf.set_index(key)

# pick all fields except OID, Geometry, and the key
fields = [
    f.name for f in arcpy.ListFields(fc)
    if f.type not in ('OID', 'Geometry') and f.name != key
]
fields.append('SHAPE@')  # token for geometry

# get the feature class’s spatial reference
sr = arcpy.Describe(fc).spatialReference

updated_ids = []
inserted_ids = []
unmatched = set(gdf_indexed.index)

# 1) Update existing rows
with arcpy.da.UpdateCursor(fc, [key] + fields) as cursor:
    for row in cursor:
        pid = row[0]
        if pid in gdf_indexed.index:
            rec = gdf_indexed.loc[pid]
            for i, fld in enumerate(fields, start=1):
                if fld == 'SHAPE@':
                    # build ArcPy geometry from WKT
                    row[i] = arcpy.FromWKT(rec.geometry.wkt, sr)
                else:
                    row[i] = rec[fld]
            cursor.updateRow(row)
            updated_ids.append(pid)
            unmatched.discard(pid)

# 2) Insert new rows for everything still unmatched
if unmatched:
    insert_fields = [key] + fields
    with arcpy.da.InsertCursor(fc, insert_fields) as icursor:
        for pid in unmatched:
            rec = gdf_indexed.loc[pid]
            new_row = [None] * len(insert_fields)
            new_row[0] = pid
            for i, fld in enumerate(fields, start=1):
                if fld == 'SHAPE@':
                    new_row[i] = arcpy.FromWKT(rec.geometry.wkt, sr)
                else:
                    new_row[i] = rec[fld]
            icursor.insertRow(new_row)
            inserted_ids.append(pid)

**Data Publishing**

In [17]:
from arcgis.gis import GIS
import time
import json

In [18]:
key='Project_ID'
agol_item_id = '196b5b06452643e4a80408d0c28b5e06'


gis = GIS(
    'https://governmentofbc.maps.arcgis.com', 
    'PX.GeoBC.DSS.Creator', 
    'maps_are_gr8!', 
    verify_cert=False

) 


print(f"..retrieving target layer with ID: {agol_item_id}")
item = gis.content.get(agol_item_id)
if not item:
    raise ValueError(f"Feature layer with ID {agol_item_id} not found.")

layer = item.layers[0]  
print(f"..found feature layer: {item.title}")

total_rows = int(arcpy.GetCount_management(fc)[0])
print(f"..source feature class contains {total_rows} rows")

print("..truncating the feature layer...")
layer.manager.truncate()
print("..feature layer truncated successfully.")

# determine which fields to upload
fields = [
    f.name for f in arcpy.ListFields(fc)
    if f.name in gdf.columns and f.type not in ("OID", "Geometry")
]

# build the search cursor
search_fields = ["SHAPE@"] + fields
failed_ids = []

print("..uploading records one by one...")
with arcpy.da.SearchCursor(fc, search_fields, "1=1") as cursor:
    for row in cursor:
        geom = row[0]
        if geom is None:
            print("..skipping record with no geometry")
            continue

        # build the attributes dict
        attrs = {fields[i]: row[i + 1] for i in range(len(fields))}
        unique_id = attrs['Project_ID']

        feature = {
            "geometry": json.loads(geom.JSON),
            "attributes": attrs
        }

        # try to add the single feature
        try:
            result = layer.edit_features(adds=[feature])
            add_res = result.get("addResults", [])
            if not add_res or not add_res[0].get("success", False):
                raise RuntimeError(add_res[0].get("error", "Unknown error"))
            print(f"..successfully added record {unique_id}")
        except Exception as e:
            print(f"..FAILED to add record {unique_id}: {e}")
            failed_ids.append(unique_id)

        # small pause to avoid hammering the service
        time.sleep(1)

if failed_ids:
    print(f"\n..upload complete with failures ({len(failed_ids)}): {failed_ids}")
else:
    print("\n..all records uploaded successfully.")


Setting `verify_cert` to False is a security risk, use at your own risk.


..retrieving target layer with ID: 196b5b06452643e4a80408d0c28b5e06
..found feature layer: Together_for_Wildlife_Funding_L1A_Feature_Layer_Project_Clusters
..source feature class contains 476 rows
..truncating the feature layer...
..feature layer truncated successfully.
..uploading records one by one...
..successfully added record 2022-T4W274
..successfully added record 2022-T4W240
..successfully added record 2022-T4W213
..successfully added record 2022-T4W271
..successfully added record 2022-T4W215
..successfully added record 2022-T4W228
..successfully added record 2022-T4W253
..successfully added record 2022-T4W242
..successfully added record 2022-T4W208
..successfully added record 2022-T4W210
..successfully added record 2022-T4W230
..successfully added record 2022-T4W264
..successfully added record 2022-T4W227
..successfully added record 2022-T4W223
..successfully added record 2022-T4W237
..successfully added record 2022-T4W205
..successfully added record 2022-T4W221
..successfully 